# Test GalvanostaticAnalyzer with Real Data

This notebook tests the `GalvanostaticAnalyzer` in `src/echemistpy/processing/analyzers/echem.py` using a real BioLogic GPCL file.

In [ ]:
import sys
import os
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

# Add src to path if running from notebooks dir
project_root = Path("..").resolve()
if str(project_root / "src") not in sys.path:
    sys.path.append(str(project_root / "src"))

from echemistpy.io.loaders import load
from echemistpy.processing.analyzers.echem import GalvanostaticAnalyzer

In [ ]:
# 1. Load Data
file_path = project_root / "docs" / "examples" / "Echem" / "Biologic_GPCL.mpt"
print(f"Loading: {file_path}")

raw_data, raw_info = load(file_path)

print("\nRaw Data Info:")
print(raw_info.to_dict())
print("\nRaw Data:")
print(raw_data.data)

In [ ]:
# 2. Analyze
analyzer = GalvanostaticAnalyzer(
    calculate_ce=True,
    ce_order="discharge",
)

# Use analyze() method which handles validation, preprocessing, computation, and metadata inheritance
result_data, result_info = analyzer.analyze(raw_data, raw_info)

# 3. Inspect Results
print("\nAnalysis Info Parameters:")
print(result_info.parameters)

print("\nResult Dataset:")
print(result_data.data)


In [ ]:
result_info.to_dict()  # Convert to dict for easier inspection

In [ ]:
# 4. Plot
fig, ax = plt.subplots(2, 1, figsize=(10, 8))
# Voltage profile
result_ds = result_data.data
if "cycle_number" in result_ds.coords:
    for cycle in result_ds.cycle_number.values:
        if cycle > 5:
            continue
        cycle_data = result_ds.sel(cycle_number=cycle)
        valid = ~np.isnan(cycle_data.time_s)
        v_key = "voltage_v" if "voltage_v" in cycle_data else "ewe_v"
        ax[0].plot(cycle_data.time_s[valid], cycle_data[v_key][valid], label=f"Cycle {cycle}")

    ax[0].set_ylabel("Voltage (V)")
    ax[0].set_xlabel("Time (s)")
    ax[0].legend()
    ax[0].set_title("Voltage Profiles per Cycle")

    # Normalized Capacity
    for cycle in result_ds.cycle_number.values:
        if cycle > 5:
            continue
        cycle_data = result_ds.sel(cycle_number=cycle)
        if "normalized_time" in cycle_data:
            valid = ~np.isnan(cycle_data.normalized_time)
            v_key = "voltage_v" if "voltage_v" in cycle_data else "ewe_v"
            ax[1].plot(cycle_data.normalized_time[valid], cycle_data[v_key][valid], label=f"Cycle {cycle}")

    ax[1].set_ylabel("Voltage (V)")
    ax[1].set_xlabel("Normalized Time")
    ax[1].set_title("Normalized Curves")
else:
    v_key = "voltage_v" if "voltage_v" in result_ds else "ewe_v"
    ax[0].plot(result_ds.time_s, result_ds[v_key])

plt.tight_layout()
plt.show()

In [ ]:
# CE Table
if "coulombic_efficiency_%" in result_ds:
    print("\nCoulombic Efficiency:")
    try:
        ce_vars = ["coulombic_efficiency_%", "charge_capacity_cal", "discharge_capacity_cal"]
        ce_ds = result_ds[ce_vars]
        ce_df = ce_ds.to_dataframe()
        print(ce_df)
    except Exception as e:
        print(f"Could not create CE DataFrame: {e}")
        print(result_ds["coulombic_efficiency_%"].values)